In [ ]:
# =============================================
# 1️⃣ Instalasi dependencies
# =============================================
!pip install trl -q
!pip install accelerate -q
!pip install transformers -q
!pip install datasets -q
!pip install huggingface_hub -q
!pip install -U -q bitsandbytes
!pip install wandb -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.6 MB/s eta 0:00:00


In [ ]:
# =============================================
# 2️⃣ Import library
# =============================================
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset
from peft import LoraConfig, get_peft_model

Mengimpor pustaka yang diperlukan untuk fine-tuning model bahasa.

*  transformers: Pustaka ini menyediakan kelas untuk memuat model pra-terlatih, tokenizer, dan konfigurasi, termasuk AutoModelForCausalLM, AutoTokenizer, AutoConfig, dan BitsAndBytesConfig yang digunakan untuk kuantisasi.
* trl: Pustaka ini, yang dirancang untuk melatih model bahasa dengan reinforcement learning, menyediakan SFTTrainer dan SFTConfig yang digunakan di sini untuk supervised fine-tuning.
* datasets: Pustaka ini digunakan untuk memuat dan memproses dataset, khususnya load_dataset untuk memuat dataset Alpaca-ID.
* peft: Pustaka ini untuk Parameter-Efficient Fine-Tuning, yang memungkinkan fine-tuning model besar dengan memori yang lebih sedikit. Pustaka ini menyediakan LoraConfig untuk mengkonfigurasi LoRA (Low-Rank Adaptation) dan get_peft_model untuk menerapkan LoRA pada model.


In [ ]:
# =============================================
# 3️⃣ Load model Phi-4 dan tokenizer
# =============================================
wandb login

In [ ]:
# =============================================
# 3️⃣ Load model Phi-4 dan tokenizer
# =============================================

model_name = "microsoft/Phi-4-mini-reasoning"

# Load config
config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
config.rope_scaling = None

# Cek expected dimensi
hidden = config.hidden_size
heads = config.num_attention_heads
partial = getattr(config, "partial_rotary_factor", 1.0)
rotary_ndims = int(hidden / heads * partial)
expected = rotary_ndims // 2

print("Expected rope_scaling length:", expected)

# Kemudian load model dengan quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=6.0)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    config=config,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

Expected rope_scaling length: 48


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

Kode ini memuat model bahasa Phi-4 dan tokenizer yang terkait.

1. Nama Model: Menentukan nama model yang akan dimuat: "microsoft/Phi-4-mini-reasoning".
2. Memuat Konfigurasi: Memuat konfigurasi model menggunakan AutoConfig.from_pretrained. trust_remote_code=True diperlukan karena model ini mungkin memerlukan kode kustom dari repositori Hugging Face. config.rope_scaling = None menonaktifkan penskalaan RoPE (Rotary Positional Embedding) jika ada.
3. Pengecekan Dimensi: Menghitung dimensi yang diharapkan untuk RoPE berdasarkan konfigurasi model. Ini adalah langkah validasi untuk memastikan konfigurasi model sesuai dengan ekspektasi.
4. Kuantisasi: Menginisialisasi konfigurasi kuantisasi menggunakan BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=6.0). Ini memungkinkan pemuatan model dalam presisi 8-bit untuk mengurangi penggunaan memori VRAM, yang penting saat bekerja dengan model besar.
5. Memuat Model: Memuat model bahasa kausal (cocok untuk tugas generasi teks) menggunakan AutoModelForCausalLM.from_pretrained. Model dimuat dengan konfigurasi yang dimodifikasi (config), konfigurasi kuantisasi (quantization_config), dan device_map="auto" yang secara otomatis mendistribusikan model ke perangkat yang tersedia (seperti GPU). trust_remote_code=True kembali digunakan di sini.
6. Memuat Tokenizer: Memuat tokenizer yang sesuai untuk model menggunakan AutoTokenizer.from_pretrained. trust_remote_code=True juga digunakan di sini.
7. Setelan Pad Token: Menetapkan pad token tokenizer ke end-of-sequence token (tokenizer.eos_token). Ini penting untuk batching data saat pelatihan, memastikan semua urutan memiliki panjang yang sama dengan menambahkan padding.

In [ ]:
# =============================================
# 4️⃣ Load & konversi dataset Alpaca-ID
# =============================================
def convert_to_messages(example):
    messages = []
    if example.get("instruction"):
        content = example["instruction"].strip()
        if example.get("input"):
            content += f"\n{example['input'].strip()}"
        messages.append({"role": "user", "content": content})
    if example.get("output"):
        messages.append({"role": "assistant", "content": example["output"].strip()})
    return {"messages": messages}

dataset = load_dataset("cahya/alpaca-id-cleaned", split="train")
dataset = dataset.map(convert_to_messages, remove_columns=dataset.column_names)

# Split train/eval
split_dataset = dataset.train_test_split(test_size=0.05, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

README.md: 0.00B [00:00, ?B/s]

alpaca-id-cleaned.jsonl:   0%|          | 0.00/44.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51590 [00:00<?, ? examples/s]

Map:   0%|          | 0/51590 [00:00<?, ? examples/s]

Kode ini berfungsi untuk memuat dan memproses dataset "cahya/alpaca-id-cleaned".

1. Fungsi convert_to_messages: Fungsi ini mengambil satu contoh dari dataset dan mengubahnya menjadi format daftar pesan yang sesuai untuk fine-tuning model berbasis chat.
    * Jika contoh memiliki kunci "instruction", ini dianggap sebagai pesan dari pengguna. Jika ada juga kunci "input", isinya akan ditambahkan ke konten instruksi.
    * Jika contoh memiliki kunci "output", ini dianggap sebagai pesan dari asisten.
    * Fungsi ini mengembalikan sebuah dictionary dengan kunci "messages" yang berisi daftar pesan yang sudah diformat.
2. Memuat Dataset: load_dataset("cahya/alpaca-id-cleaned", split="train") memuat bagian 'train' dari dataset "cahya/alpaca-id-cleaned" dari Hugging Face Hub.
3. Mengubah Format Dataset: dataset.map(convert_to_messages, remove_columns=dataset.column_names) menerapkan fungsi convert_to_messages ke setiap contoh dalam dataset. Ini mengubah struktur dataset agar setiap contoh memiliki format pesan yang diinginkan. remove_columns=dataset.column_names menghapus kolom asli dari dataset setelah konversi.
4. Memecah Dataset Menjadi Train/Eval: Dataset yang sudah dikonversi kemudian dibagi menjadi dua bagian:
    * train_dataset: Berisi 9500 contoh pertama dari dataset asli, digunakan untuk melatih model.
    * eval_dataset: Berisi 500 contoh berikutnya (dari indeks 9500 hingga 9999), digunakan untuk mengevaluasi kinerja model selama pelatihan.

In [ ]:
# =============================================
# 5️⃣ (Opsional) Gunakan LoRA untuk hemat VRAM
# =============================================
use_lora = True

if use_lora:
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["qkv_proj", "o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, lora_config)

Kode ini bersifat opsional dan digunakan untuk menghemat memori VRAM saat melakukan fine-tuning model besar dengan menggunakan metode LoRA (Low-Rank Adaptation).

1. use_lora = True: Ini adalah flag yang menentukan apakah LoRA akan digunakan atau tidak. Jika diatur ke True, blok kode di bawahnya akan dieksekusi. Jika diatur ke False, LoRA tidak akan diterapkan.
2. if use_lora:: Memeriksa apakah use_lora bernilai True.
3. lora_config = LoraConfig(...): Jika use_lora adalah True, objek LoraConfig dibuat untuk mengkonfigurasi parameter LoRA:
    * r=16: Menentukan rank dari matriks LoRA yang ditambahkan. Nilai yang lebih tinggi dapat meningkatkan kapasitas model tetapi juga meningkatkan penggunaan memori.
    * lora_alpha=32: Faktor penskalaan untuk bobot LoRA.
    * target_modules=["qkv_proj", "o_proj"]: Menentukan modul (lapisan) dalam model di mana LoRA akan diterapkan. Dalam kasus ini, LoRA diterapkan pada proyeksi query, key, value (qkv_proj) dan proyeksi output (o_proj).
    * lora_dropout=0.05: Menerapkan dropout pada lapisan LoRA untuk regularisasi.
    * bias="none": Menentukan bagaimana bias ditangani dalam LoRA. "none" berarti tidak ada bias yang ditambahkan.
    * task_type="CAUSAL_LM": Menentukan jenis tugas yang sedang dilakukan, dalam hal ini Causal Language Modeling (pemodelan bahasa kausal).
4. model = get_peft_model(model, lora_config): Mengambil model yang sudah dimuat sebelumnya dan menerapkan konfigurasi LoRA (lora_config) padanya menggunakan fungsi get_peft_model dari pustaka peft. Ini menghasilkan model baru yang di-wrapped dengan lapisan LoRA, memungkinkan fine-tuning hanya pada parameter LoRA yang ditambahkan daripada seluruh model, sehingga menghemat memori VRAM.

In [ ]:
# =============================================
# 6️⃣ Konfigurasi training
# =============================================
sft_config = SFTConfig(
    output_dir="./phi4-mini-sft-alpacaid",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=2e-5,
    warmup_ratio=0.05,
    logging_steps=25,
    save_steps=500,
    eval_steps=500,
    lr_scheduler_type="cosine",
    fp16=True,
    bf16=False,
    gradient_checkpointing=True,
    report_to="wandb",
    run_name="phi4-mini-sft-alpacaid",
    push_to_hub=False,
    max_grad_norm=1.0,
    optim="adamw_torch_fused",
    dataset_kwargs={
        "max_seq_length": 2048,
        "packing": True,
        "skip_prepare_dataset": False
    },
)

In [ ]:
# =============================================
# Inisialisasi Fungsi Formatting
# =============================================
def formatting_func(example):
    """
    Gabungkan message menjadi teks mentah yang akan ditokenisasi otomatis oleh TRL.
    Format mirip chat log (bisa kamu ubah sesuai preferensi).
    """
    text = ""
    for msg in example["messages"]:
        role = msg["role"]
        content = msg["content"]
        if role == "user":
            text += f"<|user|>: {content}\n"
        elif role == "assistant":
            text += f"<|assistant|>: {content}\n"
    return text.strip()

In [ ]:
# =============================================
# 7️⃣ Inisialisasi Trainer
# =============================================
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    formatting_func=formatting_func,
    args=sft_config,
)

Applying formatting function to train dataset:   0%|          | 0/9500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/9500 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/9500 [00:00<?, ? examples/s]

Applying formatting function to eval dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# =============================================
# 8️⃣ Jalankan training
# =============================================
trainer.train()

Step,Training Loss
25,5.499400
50,5.115400
75,4.652600
100,4.074700
125,3.233700
150,2.738300
175,2.330200
200,2.254600
225,2.101500
250,2.020500


KeyboardInterrupt: 

In [ ]:
# =============================================
# 9️⃣ Simpan hasil model fine-tuning
# =============================================
trainer.save_model("./phi4-sft-alpacaid-final")
tokenizer.save_pretrained("./phi4-sft-alpacaid-final")

print("✅ Fine-tuning selesai! Model tersimpan di ./phi4-sft-alpacaid-final")

✅ Fine-tuning selesai! Model tersimpan di ./phi4-sft-alpacaid-final


Proyek ini bertujuan untuk melakukan fine-tuning (penyetelan halus) model bahasa causal (generatif) bernama Phi-4-mini-reasoning dari Microsoft menggunakan dataset Alpaca-ID yang sudah dibersihkan.

Berikut adalah langkah-langkah utama yang dilakukan dalam proyek ini:

1. Instalasi Dependencies: Menginstal pustaka Python yang diperlukan seperti trl, accelerate, transformers, datasets, huggingface_hub, dan bitsandbytes untuk fine-tuning model bahasa.
2. Import Library: Mengimpor kelas dan fungsi spesifik dari pustaka yang diinstal untuk memuat model, tokenizer, konfigurasi, dan menyiapkan trainer.
3. Load Model dan Tokenizer: Memuat model Phi-4-mini-reasoning dan tokenizer terkait dari Hugging Face Hub. Konfigurasi model disesuaikan (misalnya, rope_scaling dinonaktifkan) dan model dimuat dengan kuantisasi 8-bit untuk mengurangi penggunaan memori VRAM.
4. Load & Konversi Dataset: Memuat dataset cahya/alpaca-id-cleaned dan mengubah formatnya menjadi format berbasis pesan (mirip format chat) yang sesuai untuk fine-tuning model causal dengan pustaka trl. Dataset kemudian dibagi menjadi set pelatihan (train) dan evaluasi (eval).
5. (Opsional) Gunakan LoRA: Menerapkan teknik LoRA (Low-Rank Adaptation) secara opsional untuk menghemat memori VRAM dengan hanya melatih subset kecil parameter yang ditambahkan ke model.
6. Konfigurasi Training: Menentukan berbagai parameter untuk proses fine-tuning menggunakan SFTConfig, termasuk direktori output, ukuran batch, learning rate, scheduler, dan pengaturan penghematan memori seperti fp16 dan gradient checkpointing.
7. Inisialisasi Trainer: Menginisialisasi objek SFTTrainer dengan model, dataset pelatihan dan evaluasi, fungsi pemformatan data, dan konfigurasi pelatihan yang telah ditentukan.
8. Jalankan Training: Memulai proses fine-tuning model menggunakan trainer.train(). Training tampaknya diinterupsi (KeyboardInterrupt) berdasarkan output yang ada.
9. Simpan Model: Menyimpan model hasil fine-tuning dan tokenizer terkait ke direktori lokal.

Secara keseluruhan, proyek ini adalah workflow untuk fine-tuning model Phi-4-mini-reasoning dalam bahasa Indonesia menggunakan dataset Alpaca-ID dengan memanfaatkan teknik penghematan memori seperti kuantisasi dan LoRA.